In [ ]:
# instal required imports

"""

import sys
!{sys.executable} -m pip install git+https://github.com/maurice-herwig/wofa    
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install ipython
!{sys.executable} -m pip install matplotlib
"""

# A Similarity Measure for Formal Languages Based on Convergent Geometric Series
###  University of Kassel, Germany

<ul>
    <li>Florian Bruse</li>
    <li><u>Maurice Herwig</u></li>
    <li>Martin Lange</li>
</ul>    

### 26th International Conference on Implementation and Application of Automata  (CIAA'22).
Rouen, June 28-July 1 2022




# WoFA 
Weight of finite automata (WoFA).
## Installation

````bash
pip install git+https://github.com/maurice-herwig/wofa
````

## Creating a Finite Automaton Object

In [ ]:
from wofa import FiniteAutomata
from wofa import weight_diff, weight
from wofa import Matrix
from ipywidgets import interactive, IntSlider, FloatSlider, Layout, ToggleButtons, HBox
from IPython.display import display, Markdown, clear_output
from matplotlib import pyplot as plt

# Setting the alphabet.
FiniteAutomata.set_alphabet({'a', 'b'})

initial_states = {1}
transitions = [(1, 'a', 2), (1, 'b', 1), (2, 'a', 2), (2, 'b', 3), (3, 'a', 3), (3, 'b', 3)]
final_states = {3}

target = FiniteAutomata(initial_states, transitions, final_states)

submission = FiniteAutomata({1}, [(1, 'a', 2), (2, 'b', 3)], {3})


In [ ]:

def share_of_word_lengths(lam, lengths, eta):
    shares = []
    fac = ((1 - lam)/ lam)
    
    sum_to_eta = 0
    
    for i in lengths:
        shares.append(lam**(i + 1) * fac)
        
        if i <= eta:
            sum_to_eta += lam**(i + 1) * fac
            
    for i in range(eta + 1):
        shares[i] = sum_to_eta / (eta + 1)
        
        
    return shares
        
    
def share_of_one_word(lam, lengths, eta):
    shares = []
    fac = ((1 - lam)/ lam)
    
    sum_w_c_p = 1 - lam ** (eta + 1)
    
    max_words_in_c_p = max_words_in_c_p = (1 - len(FiniteAutomata.get_alphabet()) ** (eta + 1)) / (1 - len(FiniteAutomata.get_alphabet()))

    w_of_one_word_c_p = sum_w_c_p / max_words_in_c_p
    

    
    for i in lengths:
        shares.append(lam**(i + 1) * fac / (len(FiniteAutomata.get_alphabet())**i))
        
            
    for i in range(eta + 1):
        shares[i] = w_of_one_word_c_p
        
    return shares
    
    
def weight_calc_lam(lam):
    lengths = [i for i in range(11)]
    
    shares = share_of_word_lengths(lam, lengths, 0)
    
    plt.bar(lengths, shares)
    
    plt.ylabel('share')
    plt.xlabel('word length')
    #plt.title('Distribution of the proportions of the share length to the total weight')
    plt.ylim([0, shares[0]+shares[0] * 0.1])
    
    
    weight = weight_diff(target, submission, 0, lam)[2]
    
    plt.show()
    l_t = '$L_{target}$'
    l_s = '$L_{submission}$'
    display(Markdown(f'<span class="outputText">Distance between {l_t} and {l_s}: {round(weight, 3)}</span>'))
    
def weight_calc_eta_and_lam(lam, eta):
    lengths = [i for i in range(11)]
    
    
    shares = share_of_one_word(lam, lengths, eta)

    plt.bar(lengths, shares)
    
    plt.ylabel('weight of one word')
    plt.xlabel('word length')
    #plt.title('Distribution of the proportions of the share length to the total weight')
    plt.ylim([0, shares[0]+ shares[0] * 0.1])
    
    weight = weight_diff(target, submission, eta, lam)[2]
    
    plt.show()
    l_t = '$L_{target}$'
    l_s = '$L_{submission}$'
    display(Markdown(f'<span class="outputText">Distance between {l_t} and {l_s}: {round(weight, 3)}</span>'))
    
def fractions(n, button):
    # geht nur für n >= 1 
    x = 21
    lam = 0.5

    transitions = [ (i, a, (i + 1) % n) for i in range(n) for a in FiniteAutomata.get_alphabet()]
    transitions.append((n , 'a', 0))

    language = FiniteAutomata({n}, transitions, {0})
    matrix = Matrix(language)


    lengths = [i for i in range(x)]
    shares = share_of_word_lengths(lam, lengths, 0)
    fraction = [matrix.get_share(i) * shares[i] for i in range(x)]


    bar1 = plt.bar(lengths, shares)
    bar2 = plt.bar(lengths, fraction)
    plt.legend([bar1, bar2], ['$\omega_{0.5}(\Sigma^n)$', '$f_{L_j}(n) \cdot \omega_{0.5}(\Sigma^n)$'], 
               prop={'size': 18})
    
    plt.ylabel('share')
    if button == 'log-scale':
        plt.yscale('log')
        
    plt.xlabel('word length')
    #plt.title('Weighting of the word lengths with inclusion of the fraction.')
    

    plt.show()
    
    display(Markdown(f'<span class="outputText">Weight of the example language: {round(weight(language, 0, 0.5), 3)}</span>'))
    
    



# Motivation

- **Distance measure between regular languages**

- For automatic feedback / grading on the solutions of students.
    - Evaluation of the <span style="color:red">semantic error</span> instead of the syntactic error. 
    
- Other fields of application:  image processing, bioinformatics, data science, etc.



    
    

<h1 style="margin-buttom: 0"> Problems with other distance measures</h1>
<h3 style="margin: 0"> Word distance  induces a distance on languages</h3>
    
 $$\hat{d}(L_1,L_2) := \min \{ d(w_1,w_2) \mid w_i \in L_i \}$$

- For typical distance types $d$: Euclidian, Manhattan, Cosine, Hamming, Levenshtein, etc.
- Ignores the <span style="color:red">inner structure</span> of these languages.

<h3 style="margin: 0">Hausdorff distance</h3>


$$ \hat{d}(L_1, L_2) = \max \big(\{ \tilde{d}(L_1, w) \mid w \in L_2 \} \cup \{ \tilde{d}(L_2, w') \mid w' \in L_1\} \big) \\
\text{ where }\tilde{d}(L,w) = \min \{d(w',w) \mid w' \in L\}
$$

<ul><li><span style="color:red">Undecidability problems</span> by using this definition ([Choffrut and Pighizzini, 2002]).
    </li></ul>

<h3 style="margin-buttom: 0">Weighting of the symmetrical difference $L_1 \triangle L_2$</h3>

<ul>
    <li>$L_1 \triangle L_2$ can be infinite.</li>
    <li><span style="color:red">Limit value</span> of the fraction of words per word length <span style="color:red">does not have to exist</span> 
        ([Alur et al., 2013]).</li>
</ul>
    
    
 


<h3 style="margin-buttom: 0">Solution</h3>
<ul>
    <li>Word lengths, <span style="color:red">descending weight</span> by using the <span style="color:red">geometric series.</span></li>
</ul>



# Weight of a Language

Let $L \subseteq \Sigma^*$ and $L^{n} = \{w \mid w \in L  $ and $|w| = n \}$. 

Let $f_L\colon\mathbb{N}\to[0,1]$ be the <span style="color:red">fraction of words</span> from $\Sigma^n$ that are in $L$.

$$f_L(n) = \frac{|L^{n}|}{|\Sigma^n|}$$

The $\lambda \in (0,1)$ <span style="color:red">weight of a language</span>
$\omega_\lambda\colon2^{\Sigma^*} \to [0,1]$ can be determined as follows. 

$$\omega_\lambda(L) = (1-\lambda) \cdot \sum_{i = 0}^{\infty} \lambda^i \cdot f_L(i) 
= (1-\lambda) \cdot \sum_{i=0}^{\infty} \lambda^i \cdot \frac{|L^{i}|}{|\Sigma^i|}$$


$\Rightarrow$ For all $\lambda \in (0,1)$: $\omega_\lambda$ is monotonic, $\omega_\lambda(\emptyset) = 0$ and $ \omega_\lambda( \Sigma^*) = 1 $.

$L_j = a\Sigma^j$ with $\Sigma = \{a, b\}$ and $\lambda = 0.5$



In [ ]:

interactive(
    fractions,
    n = IntSlider(value= 2, min= 1, max= 10, layout=Layout(width='500px'), description=r'\(\ j\)'),
    button = ToggleButtons(options=['lin-scale', 'log-scale'], description=" "),
)


    

# For a DFA the weight is computable

<span class="Theorem">**Theorem.**  $\omega_\lambda(L(\mathcal{A}))$ is computable if $\mathcal{A} = (Q, \Sigma, \delta, q_i, Q_F)$ is a  <span style="color:red">DFA.</span> </span>


**Proof-idea:**

For $q \in Q$, let $L_q = L(\mathcal{A}_q)$ with $\mathcal{A}_q = (Q, \Sigma, \delta, q,$ $Q_F)$ and the weight. 

$$\omega_\lambda(L_q) = (1-\lambda) \cdot \sum_{i=0}^{\infty} \lambda^i \cdot \frac{|L_q^{i}|}{|\Sigma^i|}\ $$ 


By using the transitions, the weights of the languages can be determined recursively.  

$$(1-\lambda) \cdot |L_q^{0}| +  \frac{\lambda}{|\Sigma|}  \cdot \sum_{a \in \Sigma} \omega_\lambda(L_{\delta(q,a)})  $$




- $ t_{q,q'}=\frac{|\{ a \in \Sigma \mid \delta(q, a) = q' \}|}{|\Sigma|}$ 
- $e_q = 1 \text{ if }q \in Q_F \text{ and }e_q = 0 \text{ otherwise } $

So we can write $\omega_\lambda(L_q)$ as:

$$\omega_\lambda(L_q) = (1-\lambda) \cdot e_q + \lambda \cdot t_{q,q_1} \omega_\lambda(L_{q_1}) + \dotsb + \lambda \cdot t_{q,q_n} \omega_\lambda(L_{q_n})$$

This results in an equation system:

$$ 
\begin{aligned}
 - (1-\lambda) \cdot e_{q_1} &=  (\lambda \cdot t_{q_1,q_1} -1) \cdot \omega_\lambda(L_{q_1}) + \dotsb +  \lambda \cdot t_{q_1,q_n}\cdot  \omega_\lambda(L_{q_n}) \\
\vdots \quad & \quad \quad \quad \quad \quad \quad \vdots \quad \quad \quad \quad \ddots\quad  \quad \quad \quad  \vdots \\
 - (1-\lambda) \cdot e_{q_n} &= \ \ \ \ \ \quad \lambda \cdot t_{q_n,q_1} \cdot \omega_\lambda(L_{q_1}) + \dotsb +  (\lambda \cdot t_{q_n,q_n}-1) \cdot \omega_\lambda(L_{q_n}) \\
\end{aligned}
$$

There is exactly a <span style="color:red">unique solution</span> for the system of equations and: 

$$\omega_\lambda(L) = \omega_\lambda(L_{q_i}) $$

$\Rightarrow \omega_\lambda(L(\mathcal{A}))$ is computable in time $\mathcal{O}(n^3)$ for a DFA with $n$ states.

# Distance of two Languages

**Aim**: weight function $\rightarrow$ distance function.

<span style="color:red">Distance of two languages</span> $d_\lambda \colon 2^{\Sigma^*} \times 2^{\Sigma^*} \to [0,1]$ 

can be determined by the <span style="color:red">symmetric difference</span> $L_1 \triangle L_2$.

$$d_\lambda(L_1, L_2) = \omega_\lambda(L_1 \triangle L_2)$$


<span  class="Theorem" >**Theorem.** $d_\lambda$ is a metric on the space of all $\Sigma$-languages.</span>

    

**Proof:**
<ul style="margin: 0;">
    <li> $d_\lambda(L_1, L_2) = 0$ iff $L_1 = L_2$: follow from $\omega_\lambda(\emptyset) = 0$.</li>
    <li>$d_\lambda(L_1, L_2) = d_\lambda(L_1, L_2)$: follow from $L_1 \triangle L_2 = L_2 \triangle L_1$.</li>
    <li>$d_\lambda(L_1, L_3) \le d_\lambda(L_1, L_2) + d_\lambda(L_2, L_3)$: follow from if $w \in L_1 \triangle L_3$ then holds $w \in L_1 \triangle L_2$ or $w \in L_2 \triangle L_3$.</li>
</ul>


# Practical application

### students evaluation

Implementation: https://github.com/maurice-herwig/wofa

# Practical application

**Recap motivation**: automatic feedback / grading  on the solutions of the students.
   
- Standard exercise: construct an automaton that recognizes exactly a given language.

- Automatic feedback initiates an <span style="color:red">error-driven learning cycle.</span>


<h1 style="margin-buttom: 0">Good parameter values?</h1>

- For feedback / grading to the students solution.

- For this we consider the difference of the following **example** languages over $\Sigma = \{a, b\}$.

$$
\begin{aligned}& L_{target} = \{w \in \Sigma^* \mid w \text{ contains the subword } ab\} \\
                & L_{submission}  = \{ab \}
\end{aligned}$$

$L_{target} = \{w \in \Sigma^* \mid w \text{ contains the subword } ab\}$ and $L_{submission} = \{ab\}$ 

In [ ]:
interactive(
    weight_calc_lam,
    lam = FloatSlider(value=0.5, min=0.1, max=0.9, layout=Layout(width='500px'), description=r'\(\lambda\)'),
)


# Redistribution of Weights on Short Words

<h3 style="margin-top: 20px">Problem $\lambda$</h3>
$\omega_\lambda$  results in a strong <span style="color:red">overweighting</span> of <span style="color:red">short word</span> lengths.

- $\eta$ near to $1$ flatten the distribution curve, but this is not a got distribution over the length. 

- Does not lead to good results.

<h3 style="margin-top: 20px">Solution: parameter $\eta$</h3>

- Words with length <span style="color:red">up to</span> a certain length $\eta$ should be weighted <span style="color:red">equally.</span>

- Word with length <span style="color:red">greater</span> than $\eta$ should be weighted <span style="color:red">exponentially decreasing.</span>

$$\omega_\lambda^\eta(L) = \omega'_\lambda(L^{(\le \eta)}) + \omega_\lambda(L^{(> \eta)})$$


$$\omega_\lambda^\eta(L) = \omega'_\lambda(L^{(\le \eta)}) + \omega_\lambda(L^{(> \eta)})$$


<span class="Theorem">**Theorem.**  $\omega_\lambda^\eta(L)$ is computable. </span>

**Proof-idea:**

- $\omega'_\lambda(L^{(\le \eta)})$ by <span style="color:red"> matrix multiplication.</span>
     - $A^1$ = Adjacency matrix of $\delta$.
     - $A^i = A^{i-1} \cdot A^1$
     - Weight of one word $\frac{1-\lambda^{\eta+1}}{\sum_{i=0}^{\eta} |\Sigma|^i}$.

- $\omega_\lambda(L^{(> \eta)}) = \omega_\lambda(L) - \omega_\lambda(L^{(\le \eta)})$

$\Rightarrow \omega'_\lambda(L^{(\le \eta)})$ is computable in time $\mathcal{O}(\eta \cdot |Q|^3)$.
    

# Example

$L = \{w \in \{a, b\}^* \mid w \text{ not contains the subword } abba\}$

In [ ]:
interactive(
    weight_calc_eta_and_lam,
    lam = FloatSlider(value=0.7, min=0.1, max=0.9, layout=Layout(width='500px'), description=r'\(\lambda\)'),
    eta = IntSlider(value= 4, min= 0, max= 10, layout=Layout(width='500px'), description=r'\(\eta\)')
)


**Good parameter values:** $\eta$ = pumping constant, $\lambda$ so that $\omega'_\lambda(\Sigma^{*^{(\le \eta)}}) = \omega_\lambda(\Sigma^{*^{(> \eta)}})$

# Summary

- <span style="color:red">Weight function</span> for regular languages.
    - Well defined by using the geometric series.
- <span style="color:red">Distance measure</span> between two regular languages.
    - By the determinations of the weight of the symmetrical difference.
- <span style="color:red">Practical use</span> for students evaluation.
    - Initial part with constant weighting of words.

<h3 style="margin-bottom: 0">Currently work</h3>

- Web page for automatic correction of student submissions by the distance measure. 

<h3 style="margin-bottom: 0">Open work</h3>

- Is it possible to calculate the distance between two NFA without explicit determination?
